# 恶意软件检测 - 随机森林分类器

本notebook使用随机森林算法对恶意软件进行检测和分类。

## 导入必要的库

In [1]:
# 数据处理和分析库
import pandas as pd
import numpy as np

# 机器学习相关库
from sklearn.model_selection import KFold  # K折交叉验证
from sklearn.utils import shuffle  # 数据随机打乱
from sklearn.ensemble import RandomForestClassifier  # 随机森林分类器
from sklearn.model_selection import train_test_split  # 训练测试集分割

## 数据集配置和加载

定义数据集的基本信息和加载数据文件。

In [2]:
# 数据集样本数量配置
DS1_BENIGN_SAMPLES_CNT = 2939    # 数据集1中良性样本数量
DS1_MAL_SAMPLES_CNT = 13734      # 数据集1中恶意样本数量
DS2_MAL_SAMPLES_CNT = 2884       # 数据集2中恶意样本数量（用于测试）

# 数据集文件名
Dataset1_name = 'ds1.xls'        # 训练数据集
Dataset2_name = 'ds2.xls'        # 测试数据集

# 加载数据集
# dataset1: 包含良性和恶意样本的训练数据集
dataset1 = pd.read_excel('/Users/wenzhuolin/workspace/macro_fc-main/ds1.xls')
# dataset2: 仅包含恶意样本的测试数据集
dataset2 = pd.read_excel('/Users/wenzhuolin/workspace/macro_fc-main/ds2.xls')

# 交叉验证配置
K_FOLD = 5  # 使用5折交叉验证

## 随机森林交叉验证函数

定义一个函数来执行K折交叉验证，评估随机森林模型的性能。

### 函数功能：
- 使用K折交叉验证训练和测试随机森林模型
- 计算每折的准确率、精确率、召回率和误报率
- 返回所有折的性能指标列表

In [3]:
def model_RF_cross_valid(XX, yy, rawfilenamelist, random_state=42):
    """
    随机森林K折交叉验证函数
    
    参数:
        XX: 特征矩阵
        yy: 标签向量 (0=良性, 1=恶意)
        rawfilenamelist: 文件名列表
        random_state: 随机种子，确保结果可重现
    
    返回:
        tuple: (精确率列表, 召回率列表, 准确率列表, 误报率列表)
    """
    # 随机打乱数据，保持特征、标签、文件名的对应关系
    X, y, filenamelist = shuffle(XX, yy, rawfilenamelist, random_state=random_state)
    
    # 初始化随机森林分类器（使用默认参数）
    rf = RandomForestClassifier()
    
    # 创建K折交叉验证对象
    kf = KFold(n_splits=K_FOLD)
    
    # 初始化性能指标列表
    accuracy_list = []   # 准确率列表
    precesion_list = []  # 精确率列表
    recall_list = []     # 召回率列表
    fp_list = []         # 误报率列表
    
    # 执行K折交叉验证
    for train_index, test_index in kf.split(X):
        # 分割训练集和测试集
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        y_test_filenamelist = filenamelist[test_index]
        
        # 训练随机森林模型
        rf.fit(X_train, y_train)
        
        # 预测测试集
        y_test_predict = rf.predict(X_test)

        # 初始化混淆矩阵计数器
        tp_cnt = 0  # True Positive: 正确识别的恶意样本
        fp_cnt = 0  # False Positive: 误报的良性样本
        tn_cnt = 0  # True Negative: 正确识别的良性样本
        fn_cnt = 0  # False Negative: 漏报的恶意样本
        
        # 计算混淆矩阵
        for i in range(len(test_index)):
            if y_test[i] == 0:  # 真实标签为良性
                if y_test_predict[i] > 0.5:  # 预测为恶意
                    fp_cnt = fp_cnt + 1  # 误报
                    # print('[FA]%s' %y_test_filenamelist[i])  # 可选：打印误报文件
                else:  # 预测为良性
                    tn_cnt = tn_cnt + 1  # 正确识别良性
            elif y_test[i] == 1:  # 真实标签为恶意
                if y_test_predict[i] > 0.5:  # 预测为恶意
                    tp_cnt = tp_cnt + 1  # 正确识别恶意
                else:  # 预测为良性
                    fn_cnt = fn_cnt + 1  # 漏报
                    # print('[UD]%s' %y_test_filenamelist[i])  # 可选：打印漏报文件
            else:
                print('Error label %f' %y_test[i])  # 错误标签警告
        
        # 计算性能指标
        accuracy = 1.0 * (tp_cnt + tn_cnt)/(tp_cnt + tn_cnt + fp_cnt + fn_cnt)  # 准确率
        precesion = 1.0 * tp_cnt / (tp_cnt + fp_cnt)  # 精确率
        recall = 1.0 * tp_cnt / (tp_cnt + fn_cnt)     # 召回率
        fp = 1.0 * fp_cnt / (tn_cnt + fp_cnt)         # 误报率
        
        # 保存当前折的性能指标
        accuracy_list.append(accuracy)
        precesion_list.append(precesion)
        recall_list.append(recall)
        fp_list.append(fp)
    
    return precesion_list, recall_list, accuracy_list, fp_list

## 数据集1的交叉验证实验

使用数据集1进行K折交叉验证，测试组合特征的检测效果。

### 特征说明：
- 第0列：文件名
- 第1-77列：混淆特征（如行长度、括号数量、过程数量等）
- 第78-123列：可疑特征（如Shell、CreateObject、Chr等API调用）
- **组合特征** (列1-123): 使用所有特征进行检测

In [4]:
print("-----------Dataset1 Cross Validation-----------")    

# 使用组合特征进行交叉验证
print("\n=== 组合特征交叉验证 ===")
# 创建标签：前DS1_BENIGN_SAMPLES_CNT个为良性(0)，后DS1_MAL_SAMPLES_CNT个为恶意(1)
labels = [0] * DS1_BENIGN_SAMPLES_CNT + [1] * DS1_MAL_SAMPLES_CNT
# 提取所有特征 (第1-123列)
XX = dataset1.iloc[:, 1:124].values
yy = np.array(labels)
rawfilenamelist = dataset1.iloc[:, 0].values  # 文件名列表

# 执行K折交叉验证
precesion, recall, accuracy, fp = model_RF_cross_valid(XX, yy, rawfilenamelist)
print('Detection result with combined features:')
print('precesion:%f, recall:%f, accuracy:%f, fp:%f' %(np.average(precesion), np.average(recall), np.average(accuracy), np.average(fp)))

-----------Dataset1 Cross Validation-----------

=== 组合特征交叉验证 ===
Detection result with combined features:
precesion:0.996797, recall:0.996580, accuracy:0.994542, fp:0.015005


## 数据集2的独立测试

使用数据集1训练模型，在数据集2上进行独立测试。
数据集2包含2884个恶意样本，用于验证模型的泛化能力。

In [5]:
# 提取数据集2的文件名列表，用于后续分析
testfilenamelist = dataset2.iloc[:, 0].values

### 组合特征在数据集2上的独立测试

In [6]:
print("-----------Dataset2 Independent Test-----------")    

# 使用组合特征进行独立测试
print("\n=== 组合特征在数据集2上的测试 ===")
# 使用数据集1的标签和所有特征训练模型
labels = [0] * DS1_BENIGN_SAMPLES_CNT + [1] * DS1_MAL_SAMPLES_CNT
XX = dataset1.iloc[:, 1:124].values  # 训练集所有特征
yy = np.array(labels)
test_X = dataset2.iloc[:, 1:124].values  # 测试集所有特征

# 训练随机森林模型
rf = RandomForestClassifier()
rf.fit(XX, yy)

# 在数据集2上进行预测
y_test_predict = rf.predict(test_X)

# 计算检测率 (Detection Rate)
# 由于数据集2全部为恶意样本，检测率 = 被正确识别为恶意的样本数 / 总样本数
DR = 1.0 * np.sum(y_test_predict) / len(y_test_predict)
print('Detection Rate with combined features:%f' %DR)
print(f'检测到恶意样本: {np.sum(y_test_predict)}/{len(y_test_predict)}')

-----------Dataset2 Independent Test-----------

=== 组合特征在数据集2上的测试 ===
Detection Rate with combined features:0.955633
检测到恶意样本: 2757/2885


### 决策树可视化

可视化随机森林中的第一棵决策树，帮助理解模型的决策过程。

In [9]:
# 决策树可视化（可选）
# 注意：需要先安装 Graphviz 系统软件
# conda install graphviz  或  brew install graphviz

try:
    # 导入可视化相关库
    import matplotlib  
    import matplotlib.pyplot as plt  
    from sklearn.tree import export_graphviz  # 决策树导出工具
    import graphviz  # 图形可视化库
      
    # 导出随机森林中第一棵决策树的结构
    dot_data = export_graphviz(
        rf.estimators_[0],  # 随机森林中的第一棵树
        out_file=None,   
        feature_names=dataset1.columns[1:124],  # 使用所有特征的列名（组合特征）
        class_names=['Benign', 'Malicious'],   # 类别名称 (注意：0=良性, 1=恶意)
        filled=True,        # 填充颜色
        rounded=True,       # 圆角矩形
        special_characters=True  # 支持特殊字符
    )    
    
    # 创建图形对象并显示
    graph = graphviz.Source(dot_data)    
    graph.view()  # 生成PDF文件并打开
    print("✅ 决策树可视化成功生成")
    
except ImportError as e:
    print(f"❌ 导入错误: {e}")
    print("💡 请安装所需库: pip install graphviz matplotlib")
except Exception as e:
    print(f"❌ 可视化失败: {e}")
    print("💡 请确保已安装 Graphviz 系统软件:")
    print("   - conda install graphviz")
    print("   - 或 brew install graphviz (macOS)")
    print("   - 或 sudo apt-get install graphviz (Ubuntu)")

ExecutableNotFound: failed to execute PosixPath('dot'), make sure the Graphviz executables are on your systems' PATH

### 分析未检测到的样本

打印出在组合特征测试中被误判为良性的恶意样本文件名。
这些样本可能具有与良性文件相似的特征组合。

In [31]:
# 打印在组合特征测试中被误判为良性的恶意样本文件名
print(f"\n组合特征测试中未检测到的恶意样本 (预测概率 < 0.5):")
undetected_count = 0
for i in range(len(y_test_predict)):
    if y_test_predict[i] < 0.5:  # 预测为良性的样本
        print(f"[{i}]{testfilenamelist[i]}")
        undetected_count += 1

print(f"\n总计未检测到: {undetected_count}/{len(y_test_predict)} 个恶意样本")
print(f"漏报率: {undetected_count/len(y_test_predict)*100:.2f}%")

[14]bebb4dea73ed3576a9d74a5ee382b029
[15]3cc6b9d45e1ed0df4c23c0722c84a063
[20]a41b8b7d4357000ebcad587759fb8d4c
[34]12c55bdf2ad5e35ba812dddacb29bab3
[35]7a603d53478baf62d207e90da0eb183f
[43]0c12f42c7b4f304810ccb8a0f5689c6d
[46]3bb2275f9e07d49457e1a1651a69b337
[53]4ae72b5cf4db3bc18d8952b38c43e8a4
[55]ed7975c7e7b1c364eb03816d4d746810
[61]2aa920a18c19fb9090cda5a138d9535a
[72]2bb7a6c351f52a2bcf874b9df4399545
[75]e484d3cef62fa913906b7bcb11357777
[76]605ebae1ed4fede243ace237268e44bb
[80]0a314450724d9312fe2bc7443786f7e7
[82]87dc4c901449b807ae1dca161c3ee0a1
[83]a491701274ac44b33f1b3551f5f7a555
[90]9f34c22d6f39cb660c9fdd9e6596c63d
[91]d00bf87fcffe7f2c478eed463c2f046a
[92]daaad49e1e3a5ce798d11327487b2d19
[104]746e8e6e7f74f244595bcf2616fe4725
[106]b2df06e09af2607a32610d0d0adb2b2e
[107]5cfca4435544a35251c0fa82b785763d
[129]bccc8343db4f6b055f75a72fa69d17c6
[130]8430c3fb87ec92f7544c7be8752063a0
[132]d7f0f0bde3c48237c04936b861bbfa3f
[133]24c8d98e97c3f1ea45e11db137654dd5
[134]352e5c03f027ed44028e181037

### 组合特征在数据集2上的独立测试

In [50]:
# 打印在可疑特征测试中被误判为良性的恶意样本
print(f"\n可疑特征测试中未检测到的恶意样本:")
undetected_suspicious = 0
for i in range(len(y_test_predict)):
    if y_test_predict[i] < 0.5:  # 预测为良性的样本
        print(f"[{i}]{testfilenamelist[i]}")
        undetected_suspicious += 1

print(f"\n可疑特征测试未检测到: {undetected_suspicious}/{len(y_test_predict)} 个恶意样本")
print(f"漏报率: {undetected_suspicious/len(y_test_predict)*100:.2f}%")

[30]6018b75cf6f7085f26c532d12ce4c52c
[40]bb5eda8ff049991c615eeb0ed0bccb91
[43]0c12f42c7b4f304810ccb8a0f5689c6d
[45]381d2bcf0a059b31f96834b7b65e0a60
[46]3bb2275f9e07d49457e1a1651a69b337
[72]2bb7a6c351f52a2bcf874b9df4399545
[73]a78560ca2888ddc6eff1a86177533289
[91]d00bf87fcffe7f2c478eed463c2f046a
[94]d544651032f47823862fd28f2b85bbf3
[95]0d43417ddb20af949dce7dd5e4b5625c
[96]3674337184a953a2bcd2121bf1176552
[97]6d1e30863b7c22f90d1c9c205197e114
[98]f9198c1a3fc21b2d59e24940a3bc6894
[99]5cce599ee9371b5c28cf7056e4ec5c5e
[140]f8c62e2972e8618cc19fdad8be411b3f
[141]2a1b2d710c4d29f770870949f1946147
[158]242de1268e550fda12899803f36c0989
[159]100a72a8ef4cf4f68df4e5e571df52bf
[186]b19f9129e7097d71e255e030c89be248
[222]fcf8077f877a1510cf6a69b0499da1f5
[249]47f7006dc586d532962011c15595100c
[255]5b900b13e1563799f5b9b83e05a0a46e
[279]c4a29d2e3591b33f324406e271eb8f8b
[299]3a7bdcb34e03164e55477fbfee9ffa88
[323]33329eef026cf8853ea52ce8dba5cfc4
[326]9908915d5b52e7262ab1fc13d845e1e8
[344]07903bdf3a214f74f703c

### 实验6: 组合特征在数据集2上的检测效果

In [50]:
# 使用组合特征进行独立测试
print("\n=== 实验6: 组合特征在数据集2上的测试 ===")
# 使用数据集1的标签和所有特征训练模型
labels = [0] * DS1_BENIGN_SAMPLES_CNT + [1] * DS1_MAL_SAMPLES_CNT
XX = dataset1.iloc[:, 1:124].values  # 训练集所有特征
yy = np.array(labels)
test_X = dataset2.iloc[:, 1:124].values  # 测试集所有特征

# 训练随机森林模型
rf = RandomForestClassifier()
rf.fit(XX, yy)

# 在数据集2上进行预测
y_test_predict = rf.predict(test_X)

# 计算检测率
DR = 1.0 * np.sum(y_test_predict) / len(y_test_predict)
print('Detection Rate with combined features:%f' %DR)
print(f'检测到恶意样本: {np.sum(y_test_predict)}/{len(y_test_predict)}')

Detection Rate with combined features:0.957019


In [28]:
# 打印在组合特征测试中被误判为良性的恶意样本
print(f"\n组合特征测试中未检测到的恶意样本:")
undetected_combined = 0
for i in range(len(y_test_predict)):
    if y_test_predict[i] < 0.5:  # 预测为良性的样本
        print(f"[{i}]{testfilenamelist[i]}")
        undetected_combined += 1

print(f"\n组合特征测试未检测到: {undetected_combined}/{len(y_test_predict)} 个恶意样本")
print(f"漏报率: {undetected_combined/len(y_test_predict)*100:.2f}%")

[20]a41b8b7d4357000ebcad587759fb8d4c
[30]6018b75cf6f7085f26c532d12ce4c52c
[43]0c12f42c7b4f304810ccb8a0f5689c6d
[45]381d2bcf0a059b31f96834b7b65e0a60
[46]3bb2275f9e07d49457e1a1651a69b337
[72]2bb7a6c351f52a2bcf874b9df4399545
[74]ff353681c7680f9496de058840a36b5d
[91]d00bf87fcffe7f2c478eed463c2f046a
[140]f8c62e2972e8618cc19fdad8be411b3f
[147]b1887eb4ae654745359091d61693382d
[158]242de1268e550fda12899803f36c0989
[159]100a72a8ef4cf4f68df4e5e571df52bf
[186]b19f9129e7097d71e255e030c89be248
[190]68f6fabe0baa4d11af48521ae85417ba
[222]fcf8077f877a1510cf6a69b0499da1f5
[276]c491101d2048ea8740a4f724e6804692
[279]c4a29d2e3591b33f324406e271eb8f8b
[323]33329eef026cf8853ea52ce8dba5cfc4
[334]c4f9a226c9ef68a0cf4ec3c4b0cb81b3
[344]07903bdf3a214f74f703c45181ef30e6
[354]10eaf8ebb0809f36ce61d03d4f2e3ac0
[355]4c6ae5d1dbd06552859bc65e04dcbf6e
[370]02a9c316af254d2705a1dc3d88978461
[374]e8b77c2a93f79c7a209d688aa05b0353
[393]737c44822f385b8d0011778e82d64f8f
[409]caa0ead9cba39e47a93a877042d40a40
[437]8d9425684628925

In [54]:
# 特征重要性分析
print("\n=== 特征重要性分析 ===")
import eli5
from eli5.sklearn import PermutationImportance

# 使用排列重要性分析特征对模型预测的影响
# 排列重要性通过随机打乱每个特征的值来测量特征的重要性
perm = PermutationImportance(rf, random_state=42).fit(test_X, [1]*len(test_X))

# 显示特征重要性权重（默认显示前20个最重要的特征）
eli5.show_weights(perm, feature_names = list(dataset1.columns[1:]))

Weight,Feature
0.0559 ± 0.0072,CreateObject
0.0519 ± 0.0049,Document_Open
0.0225 ± 0.0057,Shell
0.0076 ± 0.0032,GetObject
0.0048 ± 0.0024,Lib
0.0047 ± 0.0027,AutoOpen
0.0044 ± 0.0007,Auto_Open
0.0042 ± 0.0004,.StartupPath
0.0028 ± 0.0016,NUM_PROC
0.0024 ± 0.0012,LINE_OPS_CNT


In [68]:
# 显示前100个最重要的特征
# 这有助于理解哪些特征对恶意软件检测最有贡献
eli5.show_weights(perm, feature_names = list(dataset1.columns[1:]), top=100)

Weight,Feature
0.0559 ± 0.0072,CreateObject
0.0519 ± 0.0049,Document_Open
0.0225 ± 0.0057,Shell
0.0076 ± 0.0032,GetObject
0.0048 ± 0.0024,Lib
0.0047 ± 0.0027,AutoOpen
0.0044 ± 0.0007,Auto_Open
0.0042 ± 0.0004,.StartupPath
0.0028 ± 0.0016,NUM_PROC
0.0024 ± 0.0012,LINE_OPS_CNT


In [63]:
top_suspicious_keywords = ['CreateObject', 'Document_Open', 'Shell', 'GetObject', 'Lib', 'AutoOpen', 'Auto_Open', '.StartupPath', 'Chr',
'Asc', 'UCase', 'Left', 'Open', 'Abs', 'Split', 'System']

In [65]:
for sk in top_suspicious_keywords:
    cnt_b = 0
    cnt_m = 0
    for v in dataset1[sk][:DS1_BENIGN_SAMPLES_CNT]:
        if v > 0:
            cnt_b = cnt_b + 1
    for v in dataset1[sk][DS1_BENIGN_SAMPLES_CNT:]:
        if v > 0:
            cnt_m = cnt_m + 1 
    ratio = 1.0 * cnt_m / (cnt_b + cnt_m)
    print('%s:%f,%f' %(sk, 1 - ratio, ratio))

CreateObject:0.027644,0.972356
Document_Open:0.004059,0.995941
Shell:0.002881,0.997119
GetObject:0.000693,0.999307
Lib:0.017176,0.982824
AutoOpen:0.001175,0.998825
Auto_Open:0.096210,0.903790
.StartupPath:0.000000,1.000000
Chr:0.023127,0.976873
Asc:0.018773,0.981227
UCase:0.341954,0.658046
Left:0.109392,0.890608
Open:0.057454,0.942546
Abs:0.273239,0.726761
Split:0.034541,0.965459
System:0.122736,0.877264


In [19]:
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(XX.shape[1]):
    print("%2d) %s:%f" % (f + 1, dataset1.columns[1:][indices[f]], importances[indices[f]]))

 1) PROCEDURE_CONCAT_CNT:0.106390
 2) LINE_CONCAT_CNT:0.082955
 3) PROCEDURE_OPS_CNT:0.082365
 4) LINE_OPS_CNT:0.064822
 5) Shell:0.063433
 6) LINE_LEN:0.055622
 7) LINE_PARENTHESE_CNT:0.050712
 8) PROCEDURE_PARENTHESE_CNT:0.050576
 9) AutoOpen:0.046824
10) LINE_CNT_SUM_PROC:0.044950
11) PROCEDURE_EQUAL_CNT:0.032608
12) Document_Open:0.031886
13) CreateObject:0.031100
14) NUM_PROC:0.030936
15) PROCEDURE_STR_CNT:0.030767
16) LINE_STR_CNT:0.027707
17) Chr:0.012525
18) Open:0.012383
19) GetObject:0.011621
20) ShowWindow:0.005606
21) Workbook_Open:0.004643
22) Lib:0.004475
23) UCase:0.004256
24) Write:0.003888
25) Mid:0.003806
26) Left:0.003579
27) .Run:0.003557
28) Environ:0.003378
29) Split:0.003117
30) CallByName:0.003085
31) Replace:0.003072
32) Abs:0.002728
33) CByte:0.002394
34) .Create:0.002367
35) Rate:0.002271
36) Windows:0.002264
37) Print:0.002257
38) Output:0.002193
39) UBound:0.002062
40) Trim:0.002044
41) CLng:0.002008
42) Val:0.001871
43) CInt:0.001830
44) Cos:0.001830
45) A

## 实验结果总结

### 数据集1交叉验证结果：
1. **混淆特征**: 精确率98.24%, 召回率99.43%, 准确率98.06%, 误报率8.34%
2. **可疑特征**: 精确率99.73%, 召回率99.47%, 准确率99.34%, 误报率1.26%
3. **组合特征**: 精确率99.67%, 召回率99.65%, 准确率99.44%, 误报率1.53%

### 数据集2独立测试结果：
1. **混淆特征**: 检测率92.51%
2. **可疑特征**: 检测率85.62%
3. **组合特征**: 检测率95.70%

### 关键发现：
- **可疑特征**在交叉验证中表现最佳，误报率最低
- **组合特征**在独立测试中表现最佳，检测率最高
- **混淆特征**单独使用时误报率较高，但在独立测试中表现稳定
- 特征重要性分析揭示了对检测最有贡献的特征

### 建议：
- 在实际部署中推荐使用**组合特征**，以获得最佳的检测效果
- 可以根据具体应用场景调整误报率和检测率的平衡
- 定期更新特征工程以应对新的恶意软件变种